# Selling historic houses for maximum revenue with help from the King County housing data

This notebook is where I cleaned the data, searched for correlations and visualised insights for use in the presentation. For more details about the project, see the [README file](./README.md).



## Scenario: advising Zachary Brooks, owner of a portfolio of historic houses

* It’s February 2016

* Zachary Brooks has a large portfolio of historical houses across King County

* He wants to sell maximum 5 properties in the next year

## Questions

* How should he sell the ones that will generate most revenue?
 
  * When is the best time to sell?
 
  * Is it worth renovating before selling?
 
  * What are the best locations to sell in?

## Approach: draw insights from past data and apply to future

The task isn't to find appropriate properties in the dataset because they have already been sold and Zachary Brooks doesn't own them.

All we can do is make generalisations about the data, and hope they apply to the next year. "Hope" being the operative word.

Start by importing all the stuff we need and changing a few settings for later on.

This includes a couple of functions defined in [my_functions.py](./my_functions/my_functions.py).

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
import os
import urllib.request, zipfile

# import all the functions defined in my_functions.py
from my_functions.my_functions import *

from matplotlib.ticker import PercentFormatter
plt.rcParams.update({ "figure.figsize" : (8, 5),"axes.facecolor" : "white", "axes.edgecolor":  "black"})
plt.rcParams["figure.facecolor"]= "w"
pd.plotting.register_matplotlib_converters()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# make Seaborn a little prettier
sns.set_theme()
sns.set_color_codes()

Import the data. Note that `parse_dates` tries to parse as many date columns as it can, but it later turns out that `yr_renovated` needs some special attention.

In [11]:
# import data

csv_path = Path("data/kingcounty/kc_house_data.csv")
zip_path = Path("data/data.zip")
if not (csv_path.is_file()):
    Path("data").mkdir(parents=True, exist_ok=True)
    zip_url = "https://geodacenter.github.io/data-and-lab/data/kingcounty.zip"
    urllib.request.urlretrieve(zip_url, zip_path)
    with zipfile.ZipFile(zip_path) as housing_zip:
        housing_zip.extractall(path="data")
    
    os.remove("data/data.zip")
df = pd.read_csv(csv_path, parse_dates=["date", "yr_built", "yr_renovated"])
df.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.000,3,1.000,1180,5650,1.000,0,0,...,7,1180,0,1955-01-01,0,98178,47.511,-122.257,1340,5650
1,6414100192,2014-12-09,538000.000,3,2.250,2570,7242,2.000,0,0,...,7,2170,400,1951-01-01,1991,98125,47.721,-122.319,1690,7639
2,5631500400,2015-02-25,180000.000,2,1.000,770,10000,1.000,0,0,...,6,770,0,1933-01-01,0,98028,47.738,-122.233,2720,8062
3,2487200875,2014-12-09,604000.000,4,3.000,1960,5000,1.000,0,0,...,7,1050,910,1965-01-01,0,98136,47.521,-122.393,1360,5000
4,1954400510,2015-02-18,510000.000,3,2.000,1680,8080,1.000,0,0,...,8,1680,0,1987-01-01,0,98074,47.617,-122.045,1800,7503


Look at all those lovely columns. It later turns out we don't need anywhere near all of them, so we can drop quite a lot.


In [ ]:
df = df.iloc[:, [0, 1, 2, 10, 14, 15, 16, 17, 18]]
df.info()

In [ ]:
# Just do describe on price and condition as it doesn't make sense for the others.
df.iloc[:, 2:4].describe()

|    | Column name   | Description   |
|---:|:--------------|:--------------|
|  1 | date          | Date of the sale             |
|  2 | price         | Price of sale |
|  3 | condition     | Condition on a scale of 1 to 5 |
| 4 | yr_built      | Year property was built |
| 5 | yr_renovated  | Year property was last renovated |
| 6 | zipcode       | Zip code |
| 7 | lat           | Latitude |
| 8 | long          | Longitude |

## Clean `yr_renovated` column

`yr_renovated` didn't get imported as a datetime object, so we need to clean it. The problem is that it contains lots of 0.0 values that can't be detected as NaN.

In [ ]:
df["yr_renovated"] = pd.to_datetime(
    df["yr_renovated"].replace("0.0", np.NaN), format="%Y.0"
    )

df["yr_renovated"].dtype

### The `price` data is heavily right-skewed

Okay, let's admit it: price is what we're interested in most. Let's see how the data is distributed with a nice box plot.

In [ ]:
fig, ax = plt.subplots(figsize=(2, 5))
sns.boxplot(ax=ax, data=df["price"])


We can see there are many outliers, so the data is heavily right-skewed.

The mean is $540k but the median is $450k.

## Where to sell?

### Highest-value zip codes

We start by finding the 5 zip codes with the highest mean price.

`get_zip_averages` is defined in `my_functions.py`. It gets the mean price for every zipcode by default, but can also get median and mode.

In [ ]:
zips = get_zip_averages(df, mean_only=False)
zips

Map the zipcodes using Plotly.

The `map_zips` function is used throughout this part of the notebook to keep the plots consistent and make it easier to make small changes in one place. It optionally writes the plots to HTML and JPEG for use e.g. in presentations.

In [ ]:
def map_zips(zips, title="Zip Codes in King's County", width=800, height=600, color="price_mean", size="price_len", save_html=False, save_jpg=False):
    """ Maps zip codes with Seaborn, optionally saves output to HTML or JPEG files too

    Args:
        zips (_type_): _description_
        title (str, optional): The title. Defaults to "Zip Codes in King's County".
        sizecount (bool, optional): Whether size is the count (i.e. number of data). Defaults to False.
        save_html (bool, optional): Whether to save to HTML. Defaults to True.
        save_jpg (bool, optional):
    """
    import plotly.express as px
    zips = zips.copy()
    zips["_prefix"] = " ("
    zips["_suffix"] = ")"
    zips["hover"] = zips.index.astype(str) + zips["_prefix"] + zips["name"] + zips["_suffix"]
    fig = px.scatter_mapbox(
        zips,
        mapbox_style="stamen-toner",
        lat="latitude",
        lon="longitude",
        color=color,
        title=title,
        hover_name="hover",
        width=width,
        height=height,
        size=size,
        zoom=8.5,
        )

    fig.show()
    if save_html or save_jpg:
        file_name = title.replace(" ", "_")
        file_name = ''.join(e for e in file_name if (e.isalnum() or e == "_")).lower()
    if save_html:
        fig.write_html(f"html/{file_name}.html")
    if save_jpg:
        pass
        fig.write_image(f"images/{file_name}.jpg", width=width, height=height)
map_zips(zips, title="Zip Codes in King's Country: Average Price", save_html=True)

From the map, we can see the point at which things get expensive is around 850k. This is well above the 75th %ile for the mean price on zip codes.

In [ ]:
print(f"The 75th percentile is: {int(round(zips['price_mean'].quantile(0.75), -3))}")

### Let's get the top 5 zipcodes


By mean

In [ ]:
top_5_zips = zips.sort_values("price_mean", ascending=False).iloc[0:5]
top_5_zips.drop(columns=["latitude", "longitude", "price_median", "price_mode"])

By median

In [ ]:
top_5_zips_median = top_5_zips = zips.sort_values("price_median", ascending=False).iloc[0:5]
top_5_zips_median.drop(columns=["latitude", "longitude", "price_mean", "price_mode"])


They're the same...

In [ ]:
# The zipcode is the index...
top_5_zips.index.equals(top_5_zips_median.index)

In [ ]:
map_zips(top_5_zips, title="Top 5 most expensive zip codes")

### Get figures (average prices) for historical buildings (built before 1945) in all zipcodes

In [ ]:
# save this for later use
historic_year = pd.to_datetime("1945-12-31", yearfirst=True)

# masks come in handy later - you can reuse them to filter, and even combine them with &, | etc.
# so you don't have to write the same queries over and over again.
# I think you'd have to be careful using them on filtered DataFrames though, because they afaik
# filter purely by order (not by index/row number) - so if the number of rows change, it will be all off.
# The index should be retained, though, so it might be possible to use that if necessary.

mask_historic = df["yr_built"] < historic_year

# Get stats historic buildings in all zips in the dataset
historical_zips = get_zip_averages(df[mask_historic], mean_only=False)
map_zips(historical_zips, title="Zip Codes by Average Price, Historical Only")

Here we can clearly see the range of prices in various zip codes.
* The cheapest neighbourhoods start at $0.2M
* The priciest range up to almost $2M

### Get top 5 zips for historical properties

By mean...

In [ ]:
top_historical_zips = historical_zips.drop(columns=["latitude", "longitude", "price_median", "price_mode"]).sort_values("price_mean", ascending=False).iloc[0:5]
top_historical_zips

And by median...

In [ ]:
top_historical_zips_median = historical_zips.sort_values("price_median", ascending=False).iloc[0:5]
top_historical_zips_median.drop(columns=["latitude", "longitude", "price_mean", "price_mode"])

In [ ]:
top_historical_zips.index.equals(top_historical_zips_median.index)

In [ ]:
len(set(top_historical_zips.index).symmetric_difference(set(top_historical_zips_median.index)))

**The zipcodes are the same, but in a different order**. So let's choose the median when we plot them on the map.

In [ ]:
map_zips(top_historical_zips_median, title="Top 5 historical zip codes by median price", color="price_median")

Are these different from the overall most expensive zip codes? They have four in common - only two are in one but not the other.

In [ ]:
set(top_historical_zips.index).symmetric_difference(top_5_zips.index)

### Only those zip codes with a minimum number of properties

You might worry that some zipcodes have very thin data. So we want to show only zip codes with a minimum number of properties. We can't be too choosy as many zipcodes have a low number of historic properties. **Let's set the threshold at 10**.

In [ ]:
top_historical_zips_min_count = historical_zips[historical_zips["price_len"] >= 10].sort_values("price_median", ascending=False).iloc[0:5]
map_zips(top_historical_zips_min_count, "Top 5 historical zip codes, n > 10", size="price_len", color="price_median")

Does this give us different results than without the n > 10 criterion?

In [ ]:
set(top_historical_zips_min_count.index).intersection(set(top_historical_zips.index))

Again, there are two that are in one but not the other.

## Conclusion: Where to sell

It's clear from the data that there are 5 zipcodes with a minimum number of sales to assess whether the data is accurate. Depending on the properties Mr Brooks possesses, the following zip codes would be advantageous:

In [ ]:
top_historical_zips_median["name"].to_frame()

## Should I sell now, or wait until autumn?

In [ ]:
def plot_monthly_prices(df, title=None, ytick_lim=1.4e6, cut_y=4e5):
    fig, ax = plt.subplots()
    df = df.copy()
    df["month"] = df["date"].dt.month
    df_month_prices = df[["month", "price"]].groupby("month").median()
    p = sns.lineplot(axes=ax, data=df_month_prices, x="month", y="price")
    plt.xticks(rotation=45)

    # calculate upper limit for y axis (ytick_lim) with padding to next 100,000 unless one is passed as argument
    if ytick_lim is None:
        pricemax = df_month_prices["price"].max()
        ytick_lim = pricemax - (pricemax // 1e5)

    ax.set_yticks(ticks=np.arange(cut_y, ytick_lim + 1e5, 1e5))
    ax.set_xticks(ticks=df_month_prices.index, labels=["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
    ax.set_title(title, fontsize=16)
    ax.set_xlabel("Month")
    ax.set_ylabel("Price")

plot_monthly_prices(df, title="Average price by month for all properties")

The above chart **with y-axis cut at 400,000** shows that there are fluctuations in the market. While the overall percentage is relatively small if plot with a full y-axis, there is nevertheless quite a lot of money at stake if the trend holds in coming years. We've set the upper limit to 1.2m for comparability with the following chart.

Also note that the months January to May are from 2015, while May to December are from 2014 (and May overlaps for a few days).

In [ ]:
# For top-five zip codes
mask_top_5_zips = df["zipcode"].isin(top_5_zips.index.to_list())
plot_monthly_prices(df[mask_top_5_zips], "Average price by month for top 5 zipcodes", ytick_lim=1.4e6, cut_y=5e5)

For the top zip-codes, price-wise it makes little difference whether we sell now or in autumn. There's a larger window of opportunity now though.

If we do renovate, there's a chance to sell in autumn, but if things take longer, the spring window is waiting just after New Year.

In [ ]:
# For top five historic zip codes
mask_top_historical_zips = df["zipcode"].isin(top_historical_zips.index.to_list())
plot_monthly_prices(df[mask_top_historical_zips], title="Average price by month for top historic zipcodes")

For the top historic zipcodes, the picture is slightly different. Autumn is about the worst time to sell!

Is this because of low number of properties in the data?

In [ ]:
mask_top_historical_zips_min_count = df["zipcode"].isin(top_historical_zips_min_count.index.to_list())
plot_monthly_prices(df[mask_top_historical_zips_min_count], "Top historic zip codes with min 10 properties")

Conclusion: historic properties sold for lowest prices in autumn, with about $200k difference between the lowest and highest points. This means it would be wise to put the properties on the market now, in order to have best chances of maximising revenue.

This assumes that putting five properties on the market won't cannibalise one another by self-competing.

## Should I renovate?

Only a small percentage of the dataset has information on renovation date. So we should be suspicious.

In [ ]:
def pc_ren(df, return_value=False):
    if not return_value:
        percent_renovated = df["yr_renovated"].count() / df.index.size * 100
        print(f"{round(percent_renovated, 2)}% of buildings have been renovated.")
    else:
        return percent_renovated

pc_ren(df)

What about historic buildings?

In [ ]:
pc_ren(df[mask_historic])

This is surprising... so we should take it with a pinch of salt.

In [ ]:
# add a column "really_renovated" – this is either the year renovated, or the year of build if NaN

not_renovated_mask = df["yr_renovated"].notna()
bad_condition_mask = df["condition"] <= 3

df["really_renovated"] = df["yr_renovated"].fillna(df["yr_built"])
df

In [ ]:
condition_renovation = df[["condition", "yr_renovated"]].groupby("yr_renovated").mean()
condition_renovation

In [ ]:
p = sns.scatterplot(condition_renovation, x="yr_renovated", y="condition")
p.set_title("Year renovated against condition")

In [ ]:
# find average condition by really_renovated year
condition_x_ren_year = df[["really_renovated", "condition"]].groupby("really_renovated").mean()
sns.scatterplot(data=condition_x_ren_year)

Does yr_renovated really add anything? Let's do the same, but for just the build year.

In [ ]:
condition_x_build_year = df[["yr_built", "condition"]].groupby("yr_built").mean()
p = sns.scatterplot(data=condition_x_build_year, x="yr_built", y="condition")
p.set_title("Year built against condition")

All the scattergraphs seem to show that the year or renovation/build is actually negatively correlated to condition. We can see that older objects generally are in better condition.

**We can't conclude very much from the data because we would expect condition to correlate with renovation... but the opposite is the case. We can assume that condition is valid, and yr_renovated is not.**

### Is there any correlation between condition and price?

In [ ]:
# Separate into price bands, group by average condition, and plot a scatter graph
def average_condition_x_price_band(df, bins=10, return_pb=False):
    df = df[["price", "condition"]].copy()
    df["price_bands"] = pd.cut(df["price"], bins=bins, labels=range(1, bins+1))
    df_pb_x_c = groupby_avg_and_len(df, use_cols=["price_bands", "condition"], groupby_col="price_bands")
    if return_pb:
        price_bands = groupby_avg_and_len(df, use_cols=["price_bands", "price"], groupby_col="price_bands")
        return (df_pb_x_c, price_bands)
    else:
        return df
condition_x_priceband, price_bands = average_condition_x_price_band(df, return_pb=True)
condition_x_priceband

In [ ]:
sns.scatterplot(data=condition_x_priceband, x=condition_x_priceband.index, y="condition_mean", size="condition_len")

It looks like there is a correlation between price and condition. Many of the price bands don't have much data. Let's make a cut-off of at least 50 properties.

In [ ]:
data = condition_x_priceband[condition_x_priceband["condition_len"]>=50].join(price_bands, how="left", rsuffix="_r")
p = sns.scatterplot(data=data, x="price_mean", y="condition_mean", size="condition_len")
p.set_title("Price bands and their average condition rating, n >= 50")

### Correlation between condition and price for historic properties?

In [ ]:
condition_x_priceband_hist, price_bands_hist = condition_x_price_band(df[mask_historic], return_pb=True)
data = condition_x_priceband_hist.join(other=price_bands, how="left", rsuffix="_r")
sns.scatterplot(data=data, x="price_mean", y="condition_mean", size="condition_len")

It seems to be the same story - let's see how much data we have for each price band

In [ ]:
price_bands_hist

We're going to have to be less choosy if we want to see data for the highest price bands and go right down to 5!

In [ ]:
data = condition_x_priceband_hist[condition_x_priceband_hist["condition_len"] >=5].join(other=price_bands, how="left", rsuffix="_r")
p = sns.scatterplot(data=data, x="price_mean", y="condition_mean", size="price_len")
p.set_title("Price band against mean condition, historic buildings, n >= 5")

### There were no properties that renovated then resold in the timeframe :(

Only 14 renovated then sold within the timeframe.

In [ ]:
# Get "duplicates" - i.e. sold at least twice
# find relisted (i.e. duplicate) properties
relisted_mask = df.index.duplicated(keep=False)

# Get those that were renovated in dataset timeframe
renovated_in_timeframe_mask = df["yr_renovated"] >= df["date"].min()

# Get those that weren't renovated in timeframe
not_renovated_in_timeframe_mask = ~renovated_in_timeframe_mask

# Only 14 renovated and sold within the timeframe
df[renovated_in_timeframe_mask]

# Exclude those that were renovated twice - only include those with an ID in both
#ids = pd.merge(left=df[renovated_in_timeframe_mask]["id"], right=df[not_renovated_in_timeframe_mask]["id"], how="inner", on="id")

# Groupby renovation status and average price
# Calculate difference
# Can do this in various segments

In [ ]:
# They went for $0.7m on average - relatively low
np.around(df[renovated_in_timeframe_mask]["price"].mean(), -3) / 1e6

### Whether to renovate: Conclusion
We can conclude that:
* condition appears to have an impact on price, when you exclude the bands with very little data
* however, the information on condition is relatively limited - it all falls within quite a small range
* the data is thin above band 5, roughly the $2.5 million price tag
* the data is very thin for historical properties. HOWEVER -> a small increase in condition seems to correlate with a large increase in price.

* Now seems to be a good time to sell, and given the uncertainty in the data, and the risk that is associated with renovating (which could take longer than expected and cost more), it makes sense to pick properties for sale that are in a reasonably good condition already (as far above 3 as possible) but not to renovate them.

## Overall conclusion

* The best districts to sell for historical buildings are: ...
* Sell now!
* Don't renovate!